In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from pathlib import Path

In [ ]:
# API Request

# Endpoint
jet_fuel_price_data = 'http://api.eia.gov/series/?api_key=6OerksEg68hMpcXhk6UrjNzXGKPBpEf6KombS4OS&series_id=PET.EER_EPJK_PF4_RGC_DPG.D'

# Fetch the data
response = requests.get(jet_fuel_price_data).json()

# Display the data
# jfl = json.dumps(response, indent = 4)

In [ ]:
print(response)

In [ ]:
type(response)

In [ ]:
response.keys()

In [ ]:
response['series']

In [ ]:
test = response['series']

In [ ]:
type(test)

In [ ]:
test

In [ ]:
type(test[0])

In [ ]:
df_jf = pd.DataFrame(test[0]['data'])

In [ ]:
df_jf

In [ ]:
df_jf.dtypes


In [ ]:
import datetime

In [ ]:
df_jf[0]

In [ ]:
df_jf[0] = pd.to_datetime(df_jf[0])

In [ ]:
df_jf.dtypes

In [ ]:
df_jf

In [ ]:
# df.rename(columns={"A": "a", "B": "c"})
df_jf.rename(columns={0 : 'date', 1 : 'jf_price'}, inplace = True)

In [ ]:
df_jf.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
save_path = Path('SAVE_5_YEAR.csv')

# Read in data and index by date
save_data = pd.read_csv(
    save_path, 
    index_col='Date',     
    parse_dates=True, 
    infer_datetime_format=True
)
save_data

In [ ]:
save_data = save_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)

In [ ]:
save_data.head()

In [ ]:
save_data.dtypes

In [ ]:
df_jf.head()

In [ ]:
df_jf.set_index('date', inplace = True)

In [ ]:
df_jf.head()

In [ ]:
save_data.head()

In [ ]:
save_data.index

In [ ]:
df_jf.index

In [ ]:
df = df_jf.join(save_data)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head(25)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.plot.area(figsize=(16, 16), subplots=True)

In [ ]:
import seaborn as sns
correlate = df.corr()
plt.figure(figsize=(15,15))
plt.title('Correlation Matrix')
sns.heatmap(correlate, vmax =1, square=True, annot=True, cmap='cubehelix')

In [ ]:
X = df[['jf_price', 'Close']].dropna().copy()

X


In [ ]:
# Display sample data
display(X.head())
display(X.tail())

In [ ]:
X.info()

In [ ]:
X

In [ ]:
y = df['Close'].copy()
y

In [ ]:
from pandas.tseries.offsets import DateOffset


In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

# Select the ending period for the training data with an offset of 3 months
training_end = training_begin + DateOffset(months=36)

# Display the training end date
print(training_end)

In [ ]:
X

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc['2017-07-17':'2020-07-17']
y_train = y.loc['2017-07-17':'2020-07-17']

# Display sample data
X_train.tail()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc['2020-07-17':'2022-07-17']
y_test = y.loc['2020-07-17':'2022-07-17']

# Display sample data
X_test.head()

In [ ]:
# Import the required module
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X_train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
"""
# Import a new classifier from SKLearn
from sklearn.linear_model import LogisticRegression

# Initiate the model instance
logistic_regression_model = LogisticRegression()
"""

In [ ]:
"""
# Fit the model using the training data
model = logistic_regression_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
pred = logistic_regression_model.predict(X_test_scaled)

# Review the model's predicted values
pred[:10]
"""

In [ ]:
# Import the SVM model
from sklearn import svm

In [ ]:
# Create the classifier model
svm_model = svm.SVR()

# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
y_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix




# Use the KNN classifier to fit data:
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(X_train, y_train) 

# Predict y data with classifier: 
y_predict = classifier.predict(X_test)

# Print results: 
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

In [ ]:
# Import the required module
from sklearn.metrics import classification_report


# Evaluate the model using a classification report
#training_report = classification_report(y_train, training_signal_predictions)
#print(training_report)

print(classification_report(y_train, training_signal_predictions))


In [ ]:
df.head()

In [116]:
changes_df = df.pct_change()
changes_df.dropna(inplace=True)

In [ ]:
changes_df.corr()

In [117]:
changes_df['Actual Returns'] = 0.0
changes_df.loc[changes_df['Close'] >= 0, 'Actual Returns'] = 1
changes_df.loc[changes_df['Close'] < 0,  'Actual Returns'] = -1
changes_df.head()

,jf_price,Close,Actual Returns
date,,,
2022-07-11,0.025298,0.009930,1.0
2022-07-08,-0.023011,0.020074,1.0
2022-07-07,-0.006810,-0.040562,-1.0
2022-07-06,-0.072571,0.019674,1.0
2022-07-05,0.050832,-0.009031,-1.0


In [118]:
X = changes_df['jf_price'].dropna().copy()

In [119]:
y = changes_df['Actual Returns'].dropna()

In [120]:
# Generate the X_train and y_train DataFrames
X_train = X.loc['2017-07-17':'2020-07-17']
y_train = y.loc['2017-07-17':'2020-07-17']

# Display sample data

X_train.tail()


date
2017-07-24   -0.033175
2017-07-21   -0.002801
2017-07-20    0.020365
2017-07-19    0.004818
2017-07-18   -0.032877
Name: jf_price, dtype: float64

In [121]:
# Generate the X_test and y_test DataFrames
X_test = X.loc['2020-07-17':'2022-07-17']
y_test = y.loc['2020-07-17':'2022-07-17']

# Display sample data
X_test.head()

C:\Users\STATIS~1\AppData\Local\Temp/ipykernel_23272/1964512869.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  X_test = X.loc['2020-07-17':'2022-07-17']
C:\Users\STATIS~1\AppData\Local\Temp/ipykernel_23272/1964512869.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  y_test = y.loc['2020-07-17':'2022-07-17']


date
2022-07-11    0.025298
2022-07-08   -0.023011
2022-07-07   -0.006810
2022-07-06   -0.072571
2022-07-05    0.050832
Name: jf_price, dtype: float64

In [122]:
# Create a StandardScaler instance
import numpy as np
scaler = StandardScaler()

X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
# Apply the scaler model to fit the X_train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [128]:
# Import a new classifier from SKLearn
from sklearn.linear_model import LogisticRegression

# Initiate the model instance
logistic_regression_model = LogisticRegression()

In [129]:

# Fit the model using the training data
model = logistic_regression_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
pred = model.predict(X_test_scaled)

In [130]:

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        -1.0       0.49      0.43      0.46       229
         1.0       0.55      0.62      0.59       263

    accuracy                           0.53       492
   macro avg       0.52      0.52      0.52       492
weighted avg       0.53      0.53      0.53       492



In [87]:
len(pred)

492